In [1]:
!pip3 install contractions sentence_transformers umap-learn

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import re

import csv
import contractions
import nltk
from functools import reduce
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sentence_transformers import SentenceTransformer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import umap.umap_ as umap

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


In [3]:
def remove_tags(text: str) -> str:
    """Removes social media tags."""
    return re.sub('@\w+', '', text.lower())

def remove_punctuations(text: str) -> str:
    """Removes punctuations and keeps all alphanumerical text."""
    return re.sub(r'[^\w\s]', '', text)

def remove_stopwords(text: str) -> str:
    """Removes stopwords from text."""
    new_text = text
    for s in stopwords.words('english'):
        new_text = re.sub(f' {s} ', ' ', new_text)
    return new_text

def lemmatize(text: str) -> str:
    """Converts a text into its lemmatized form."""
    wnl = WordNetLemmatizer()
    return ' '.join([wnl.lemmatize(word) for word in text.split()])

In [4]:
functions = [
    remove_tags,
    contractions.fix,
    remove_punctuations,
    remove_stopwords,
    lemmatize, 
]

mass_apply = lambda x: reduce(lambda y, f: f(y), functions, x)

In [6]:
df = pd.read_excel('datasets/thesmartlocal.xlsx')
embedder = SentenceTransformer('all-mpnet-base-v2')


In [ ]:
corpus_preview = list(df['preview'].apply(mass_apply))
corpus_preview_embeddings = embedder.encode(corpus_preview)

preview_embeddings_df = pd.DataFrame(corpus_preview_embeddings)
preview_embeddings_df.to_csv('/content/preview_embeddings_BERT.csv')

In [ ]:
corpus_content = list(df['content'].apply(mass_apply))
corpus_content_embeddings = embedder.encode(corpus_content)

content_embeddings_df = pd.DataFrame(corpus_content_embeddings)
content_embeddings_df.to_csv('/content/content_embeddings_BERT.csv')

In [ ]:
preview_embeddings_df = pd.read_csv('/content/preview_embeddings_BERT.csv')
content_embeddings_df = pd.read_csv('/content/content_embeddings_BERT.csv')

In [ ]:
n_components = 10
umap_model = umap.UMAP(n_components=n_components)

corpus_preview_umap = umap_model.fit_transform(corpus_preview_embeddings)
corpus_preview_umap

array([[16.70693   ,  7.820575  ,  9.057797  , ...,  8.570907  ,
        11.934839  , -0.53622854],
       [16.560272  ,  8.376318  ,  9.047758  , ...,  9.200884  ,
        12.34353   ,  0.39384997],
       [16.805668  ,  8.114635  ,  9.625596  , ...,  9.771258  ,
        12.619641  ,  0.42503592],
       ...,
       [16.891914  ,  8.716705  ,  9.708004  , ...,  8.917812  ,
        12.700037  , -0.43307   ],
       [16.70019   ,  8.110684  ,  9.87915   , ...,  9.594297  ,
        12.807452  , -0.19301294],
       [17.473438  ,  8.158591  ,  8.869604  , ...,  8.605737  ,
        13.183232  , -0.24619396]], dtype=float32)